# Time Efficient Queries on a CSV File

In this project we aim to build fast queries that can be excuted on a CSV file in order to answer questions and conclude insights about the data being investigated.

Our approach is to create a class that represents our inventory, the class methodes represent the different queries we want to execute and in order to provide fast execution; preprocessing of the data will be implemented as well as using logarithmic search on sorted data.

For each of the following queries we will implement two different methods and later we will analyis both with respect to execution time.

- Get laptop from ID.
- Given an amount of money, find whether there are one or sum of two laptops which have price equal to that given amount
- Identify all laptops whose price falls within a given budget.



## Dataset

For this project, the dataset `laptops.csv` will be used. It contains information about 1300 laptop models.
The dataset can be downloaded from (here)[https://www.kaggle.com/ionaskel/laptop-prices/download].

Following are the 13 different columns in our dataset:
- __ID__ : Unique identifier for the laptop.
- __Company__ : Manufacturer company name.
- __Product__ : Product name.
- __TypeName__ : Laptop type.
- __Inches__ : Screen size in inches.
- __ScreenResolution__ : Screen resolution in pixels.
- __Cpu__ : CPU model.
- __Ram__ : RAM amount.
- __Memory__ : Hard drive size.
- __Gpu__ : GPU model.
- __OpSys__ : Operating system.
- __Weight__ : Laptop weight.
- __Price_euros__ : Price in Euros.

## Dataset Inspection

Next we check for the encoding used by using the `chardet` module on the raw byte data:


In [1]:
import chardet

with open("/Users/abdallarashwan/Documents/Python Projects/Datasets/laptops.csv" , 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


We start by reading the data from the CSV file using the `CSV` module by specifing the encoding `ISO-8859-1` and save it as a list in order to have a first look at the dataset.


In [2]:
import csv

with open("/Users/abdallarashwan/Documents/Python Projects/Datasets/laptops.csv" , encoding = 'ISO-8859-1' ) as file:   # use absolute path of the downloaded CSV file
    reader = csv.reader(file)
    rows = list(reader)
    header = rows[0]
    rows = rows[1:]

In [3]:
print(header)

['', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']


In [4]:
print(rows[:5])

[['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69'], ['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94'], ['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575.00'], ['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45'], ['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Plus Graphics 650', 'macOS', '1.37kg', '1803.60']]


## Class Implementation 

In this step we start implementing our class `Inventory` which has the following parameters:
- __header__ : Header from the dataset as list.
- __rows__ : laptop data as a list of lists.
- __row_from_id__ : Dictionary of rows with ID as the key.


In [5]:
class Inventory():
    def __init__(self , file_name):
        with open(file_name , encoding = 'ISO-8859-1' ) as file:
            reader = csv.reader(file)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1]) 
            
        self.row_from_id = {}
        self.prices = set()
        for row in self.rows:
            self.row_from_id[row[0]] = row
            self.prices.add(row[-1])
            
        self.rows_sorted_by_price = sorted(self.rows)
    
               
            
    def get_laptop_from_id(self , laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
            
    def get_laptop_from_id_preprocessing(self , laptop_id):
        if laptop_id in self.row_from_id:
            return self.row_from_id[laptop_id]
        else:
            return None
        
    def check_promotion_dollars(self , amount):
        for row in self.rows:
            if row[-1] == amount :
                return True
        for r1 in self.rows:
            for r2 in self.rows:
                if r1[-1] + r2[-1] == amount:
                    return True
        return False
    
    def check_promotion_dollars_preprocessing(self , amount):
        if amount in self.prices:
            return True
        for price in self.prices:
            if amount - price in self.prices:
                return True
        return False
    
    def find_laptop_by_price_binary_search(self , target_price):
        for row in self.rows:
            if row[-1] == target_price:
                return row
        return -1
    
    def find_laptop_by_price_log_search(self , target_price):
        range_start = 0
        range_end = len(self.rows_sorted_by_price) - 1
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2
            value = self.rows_sorted_by_price[range_middle][-1]
            if value == target_price:
                return self.row_from_id[range_middle]
            elif value < target_price:
                range_start = range_middle + 1
            else:
                range_end = range_middle - 1
        if self.rows_sorted_by_price[range_start][-1] != target_price:
            return -1
        return self.rows_sorted_by_price[range_start]
    
    

In [6]:
inventory = Inventory("/Users/abdallarashwan/Documents/Python Projects/Datasets/laptops.csv")
print(inventory.header)
print(len(inventory.rows))

['', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']
1303


## Class methods testing

In [7]:
print(inventory.get_laptop_from_id('352'))

['352', 'HP', 'Stream 14-AX001nv', 'Notebook', '14', '1366x768', 'Intel Celeron Dual Core N3060 1.6GHz', '2GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '1.44kg', 279.0]


In [8]:
print(inventory.get_laptop_from_id_preprocessing('1320'))

['1320', 'Asus', 'X553SA-XX031T (N3050/4GB/500GB/W10)', 'Notebook', '15.6', '1366x768', 'Intel Celeron Dual Core N3050 1.6GHz', '4GB', '500GB HDD', 'Intel HD Graphics', 'Windows 10', '2.2kg', 369.0]


In [9]:
print(inventory.check_promotion_dollars(1000))

True


In [10]:
print(inventory.check_promotion_dollars(100))

False


In [11]:
print(inventory.check_promotion_dollars_preprocessing(1000))

True


In [12]:
print(inventory.check_promotion_dollars_preprocessing(100))

False


In [13]:
print(inventory.find_laptop_by_price_binary_search(1026.00))

['857', 'Asus', 'ZenBook UX310UQ-GL026T', 'Ultrabook', '13.3', 'IPS Panel Full HD 1920x1080', 'Intel Core i5 6200U 2.3GHz', '8GB', '512GB SSD', 'Nvidia GeForce 940M', 'Windows 10', '1.45kg', 1026.0]


In [14]:
print(inventory.find_laptop_by_price_binary_search(10000))

-1


In [15]:
print(inventory.find_laptop_by_price_log_search(1026))

['857', 'Asus', 'ZenBook UX310UQ-GL026T', 'Ultrabook', '13.3', 'IPS Panel Full HD 1920x1080', 'Intel Core i5 6200U 2.3GHz', '8GB', '512GB SSD', 'Nvidia GeForce 940M', 'Windows 10', '1.45kg', 1026.0]


In [24]:
print(inventory.find_laptop_by_price_log_search(10000))

-1


## Analysing get laptop from ID methods

Next we analyis the two implemented methods for getting the laptop from ID.

We do so by calculating the total time needed (in seconds) to get 1000 laptops.
Using the time module we can calculate the execution time for each call of the method
The random module will be used to generate the list of IDs needed for the analysis.



In [16]:
import random
import time

IDs = [str(random.randint(0 , 1300)) for _ in range(1001)]

total_time = 0
for ID in IDs:
    start = time.time()
    inventory.get_laptop_from_id(ID)
    end = time.time()
    total_time += end - start
    
total_time_preprocessing = 0
for ID in IDs:
    start = time.time()
    inventory.get_laptop_from_id_preprocessing(ID)
    end = time.time()
    total_time_preprocessing += end - start
    


In [26]:
print("Without preprocessing : ",total_time)
print("With preprocessing : ",total_time_preprocessing)
print(total_time // total_time_preprocessing , " times faster")

Without preprocessing :  0.04925704002380371
With preprocessing :  0.0004227161407470703
116.0  times faster


## Analysing Promotion methods

Similarly we compare the two implemented methods as follows:


In [18]:
prices = [random.randint( 100 , 5000) for _ in range(1001)]

total_time_promotion = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_promotion += end - start
    
total_time_promotion_preprocessing = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_preprocessing(price)
    end = time.time()
    total_time_promotion_preprocessing += end - start
    

In [25]:
print("Without preprocessing : ",total_time_promotion)
print("With preprocessing : ",total_time_promotion_preprocessing)
print(total_time_promotion // total_time_promotion_preprocessing , " times faster")

Without preprocessing :  16.387206315994263
With preprocessing :  0.011806249618530273
1388.0  times faster


## Analysing find laptop by price methods

In [23]:
start = time.time()
inventory.find_laptop_by_price_binary_search(1024)
end = time.time()
t1 = end - start

start = time.time()
inventory.find_laptop_by_price_log_search(1024)
end = time.time()
t2 = end - start

print("Using binary search : ",t1)
print("Using log search : ",t2)
print(t1 // t2 , " times faster")

Using binary search :  0.00024390220642089844
Using log search :  6.604194641113281e-05
3.0  times faster
